In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import sklearn
from statsmodels.tsa.arima_model import ARIMA
import yfinance


In [3]:
raw_data = yfinance.download(tickers = '^GSPC ^FTSE ^N225 ^GDAXI'
,start="1994-01-07",end = "2018-01-29", interval = "1d",group_by="ticker",
auto_adjust=True,treads = True)

[*********************100%***********************]  4 of 4 completed


In [4]:
data_copy = raw_data.copy()

In [5]:
data_copy['spx'] = data_copy['^GSPC'].Close[:]
data_copy['dax'] = data_copy['^GDAXI'].Close[:]
data_copy['ftse'] = data_copy['^FTSE'].Close[:]
data_copy['nikkei'] = data_copy['^N225'].Close[:]

In [6]:
data_copy = data_copy.iloc[1:]
data_copy.drop(['^GSPC','^N225','^GDAXI','^FTSE'],axis = 1,inplace = True)
data_copy = data_copy.asfreq('b')
data_copy = data_copy.fillna(method = 'ffill')

In [7]:
data_copy['returns_spx'] = data_copy.spx.pct_change(1)*100
data_copy['returns_ftse'] = data_copy.ftse.pct_change(1)*100
data_copy['returns_dax'] = data_copy.dax.pct_change(1)*100
data_copy['returns_nikkei'] = data_copy.nikkei.pct_change(1)*100

In [8]:
size = int(len(data_copy)*0.8)
data_copy_train,data_copy_test = data_copy.iloc[:size],data_copy.iloc[size:]

### Fitting Auto ARIMA model

In [9]:
from pmdarima import auto_arima

In [10]:
auto_arima_model = auto_arima(data_copy_train.returns_ftse[1:])

In [11]:
auto_arima_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 5019
Model:               SARIMAX(3, 0, 5)   Log Likelihood               -7883.727
Date:                Mon, 10 Aug 2020   AIC                          15785.453
Time:                        10:52:23   BIC                          15844.142
Sample:                             0   HQIC                         15806.019
                               - 5019                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.1273      0.078     -1.626      0.104      -0.281       0.026
ar.L2         -0.8760      0.020    -44.845      0.000      -0.914      -0.838
ar.L3         -0.2776      0.072     -3.879      0.000      -0.418      -0.137
ma.L1          0.1026      0.078      1.311      0.190      -0.051       0.256
ma.L2          0.8224      0.023     36.517      0.000       0.778       0.867
ma.L3          0.1648      0.070      2.362      0.018       0.028       0.302
ma.L4         -0.0085      0.009     -0.917      0.359      -0.027       0.010
ma.L5         -0.1114      0.008    -13.437      0.000      -0.128      -0.095
sigma2         1.3547      0.015     91.102      0.000       1.326       1.384
===================================================================================
Ljung-Box (Q):                       70.94   Jarque-Bera (JB):              6244.41
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.99   Skew:                            -0.17
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.45
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""